In [1]:
import os

In [2]:
%pwd

'e:\\My_Works\\Internships\\iNeuron\\mushroom_classification\\test'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\My_Works\\Internships\\iNeuron\\mushroom_classification'

Crate the entity

In [5]:
# Crate the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

update the configuration manager in src config

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

update the components

In [8]:
import os
import urllib.request as request
import zipfile
from src import logger
from src.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> None:
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(
                f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {
                        get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self) -> None:
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

update the pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-24 11:23:20,571: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-24 11:23:20,572: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-24 11:23:20,575: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-24 11:23:20,577: INFO: common: created directory at: artifacts]
[2024-04-24 11:23:20,578: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-24 11:23:20,579: INFO: 1549246017: File already exists of size: ~ 27 KB]
